In [11]:
from selenium import webdriver  # type: ignore
from selenium.webdriver.common.by import By  # type: ignore
from selenium.webdriver.support.ui import WebDriverWait  # type: ignore
from selenium.webdriver.support import expected_conditions as EC  # type: ignore
import time
import pandas as pd  # type: ignore
import re  # type: ignore

# Initialize the WebDriver
driver = webdriver.Chrome()

try:
    # Open the RedBus website and maximize the window
    driver.get('https://www.redbus.in/')
    driver.maximize_window()
    time.sleep(5)  # Adjust if necessary

    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 20)

    # Find and click the element to get to the first page of bus routes
    element = driver.find_element(By.XPATH, "/html/body/section/div[2]/main/div[3]/div[3]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]")
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(2)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(10)

    # Initialize lists to store route data
    route_names = []
    route_links = []

    while True:
        # Extract bus route names and links
        try:
            orgname_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            for element in orgname_elements:
                route_names.append(element.text)
                route_links.append(element.get_attribute('href'))
        except Exception as e:
            print(f"Error extracting route data: {e}")
            break

        # Find and handle pagination
        try:
            pagination_container = driver.find_element(By.CLASS_NAME, "pagination")
            page_tabs = pagination_container.find_elements(By.CLASS_NAME, "page")

            # Find the active page and determine if there is a next page
            active_page = pagination_container.find_element(By.CLASS_NAME, "active")
            active_page_number = int(active_page.text)

            next_page_number = active_page_number + 1
            next_page = None
            for page in page_tabs:
                if page.text.isdigit() and int(page.text) == next_page_number:
                    next_page = page
                    break

            if next_page:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page)
                time.sleep(2)  # Wait for the scroll to complete
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(5)  # Adjust sleep time if necessary
            else:
                break
        except Exception as e:
            print(f"Error handling pagination: {e}")
            break

    # Function to extract bus details from each route page
    def extract_bus_details(route_link):
        driver.get(route_link)
        time.sleep(10)
        scroll_and_load(driver)

        try:
            orgname = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
            departure_time = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_time = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
            arrival_time = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
            amount_elements = driver.find_elements(By.CSS_SELECTOR, 'div.fare.d-block > span.f-19.f-bold')
            rating_k = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
            seat = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")
            bustype = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")

            def extract_integer(text):
                match = re.search(r'\d+', text)
                return int(match.group()) if match else 0

            def extract_star_rating(text):
                match = re.search(r'\d+', text)
                return float(match.group()) if match else 0

            orglist = [i.text for i in orgname]
            depart_time = [i.text for i in departure_time]
            duration_t = [i.text for i in duration_time]
            arrival_time = [i.text for i in arrival_time]
            amounts = [i.text for i in amount_elements]
            ratings_k = [extract_star_rating(rating.text) for rating in rating_k]
            seatavailable = [extract_integer(seat.text) for seat in seat]
            bus_type = [i.text for i in bustype]

            min_length = min(len(orglist), len(depart_time), len(duration_t), len(arrival_time), len(amounts), len(ratings_k), len(seatavailable), len(bus_type))

            bus_details = []
            if min_length > 0:
                for index in range(min_length):
                    bus_details.append({
                        'orgname': orglist[index] if index < len(orglist) else None,
                        'depart_time': depart_time[index] if index < len(depart_time) else None,
                        "duration": duration_t[index] if index < len(duration_t) else None,
                        "arrival": arrival_time[index] if index < len(arrival_time) else None,
                        'amount': amounts[index] if index < len(amounts) else None,
                        "rating": ratings_k[index] if index < len(ratings_k) else None,
                        "seat": seatavailable[index] if index < len(seatavailable) else None,
                        "bustype": bus_type[index] if index < len(bus_type) else None,
                        "route_link": route_link  # Add route link here
                    })
            else:
                print("No data to process.")
            
            return bus_details
        except Exception as e:
            print(f"Error extracting bus details from route: {e}")
            return []

    # Scroll and load all elements on the page
    def scroll_and_load(driver, scroll_pause_time=2):
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    # Extract details for each route
    all_bus_details = []
    for link in route_links:
        print(f"Processing route: {link}")
        route_details = extract_bus_details(link)
        all_bus_details.extend(route_details)
        print(f"Extracted {len(route_details)} bus details from the route.")

    # Create a DataFrame and save to CSV
    df = pd.DataFrame(all_bus_details)
    print(df)
    df.to_csv('bus_details.csv', index=False)

finally:
    # Close the WebDriver
    driver.quit()


Error handling pagination: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pagination"}
  (Session info: chrome=127.0.6533.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7C6879632+30946]
	(No symbol) [0x00007FF7C682E3C9]
	(No symbol) [0x00007FF7C6726FDA]
	(No symbol) [0x00007FF7C677822C]
	(No symbol) [0x00007FF7C677850C]
	(No symbol) [0x00007FF7C67BDCB7]
	(No symbol) [0x00007FF7C679CAAF]
	(No symbol) [0x00007FF7C67BB041]
	(No symbol) [0x00007FF7C679C813]
	(No symbol) [0x00007FF7C676A6E5]
	(No symbol) [0x00007FF7C676B021]
	GetHandleVerifier [0x00007FF7C69AF83D+1301229]
	GetHandleVerifier [0x00007FF7C69BBDB7+1351783]
	GetHandleVerifier [0x00007FF7C69B2A03+1313971]
	GetHandleVerifier [0x00007FF7C68ADD06+245686]
	(No symbol) [0x00007FF7C683758F]
	(No symbol) [0x00007FF7C6833804]
	(No symbol) [0x00007FF7C683

In [17]:
from selenium import webdriver  # type: ignore
from selenium.webdriver.common.by import By  # type: ignore
from selenium.webdriver.support.ui import WebDriverWait  # type: ignore
from selenium.webdriver.support import expected_conditions as EC  # type: ignore
import time
import pandas as pd  # type: ignore
import re  # type: ignore

# Initialize the WebDriver
driver = webdriver.Chrome()

try:
    # Open the RedBus website and maximize the window
    driver.get('https://www.redbus.in/')
    driver.maximize_window()
    time.sleep(5)  # Adjust if necessary

    # Initialize WebDriverWait
    wait = WebDriverWait(driver, 20)

    # Find and click the element to get to the first page of bus routes
    element = driver.find_element(By.XPATH, "/html/body/section/div[2]/main/div[3]/div[3]/div[2]/div/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]")
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(2)
    driver.execute_script("arguments[0].click();", element)
    time.sleep(10)

    # Initialize lists to store route data
    route_names = []
    route_links = []

    while True:
        # Extract bus route names and links
        try:
            orgname_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
            for element in orgname_elements:
                route_names.append(element.text)
                route_links.append(element.get_attribute('href'))
        except Exception as e:
            print(f"Error extracting route data: {e}")
            break

        # Find and handle pagination
        try:
            pagination_container = driver.find_element(By.CLASS_NAME, "pagination")
            page_tabs = pagination_container.find_elements(By.CLASS_NAME, "page")

            # Find the active page and determine if there is a next page
            active_page = pagination_container.find_element(By.CLASS_NAME, "active")
            active_page_number = int(active_page.text)

            next_page_number = active_page_number + 1
            next_page = None
            for page in page_tabs:
                if page.text.isdigit() and int(page.text) == next_page_number:
                    next_page = page
                    break

            if next_page:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page)
                time.sleep(2)  # Wait for the scroll to complete
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(5)  # Adjust sleep time if necessary
            else:
                break
        except Exception as e:
            print(f"Error handling pagination: {e}")
            break

    # Function to extract bus details from each route page
    def extract_bus_details(route_link):
        driver.get(route_link)
        time.sleep(10)
        scroll_and_load(driver)

        try:
            orgname = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
            departure_time = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_time = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
            arrival_time = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
            amount_elements = driver.find_elements(By.CSS_SELECTOR, 'div.fare.d-block > span.f-19.f-bold')
            rating_k = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
            seat = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")
            bustype = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")

            def extract_integer(text):
                match = re.search(r'\d+', text)
                return int(match.group()) if match else 0

            def extract_star_rating(text):
                match = re.search(r'\d+', text)
                return float(match.group()) if match else 0

            orglist = [i.text for i in orgname]
            depart_time = [i.text for i in departure_time]
            duration_t = [i.text for i in duration_time]
            arrival_time = [i.text for i in arrival_time]
            amounts = [i.text for i in amount_elements]
            ratings_k = [extract_star_rating(rating.text) for rating in rating_k]
            seatavailable = [extract_integer(seat.text) for seat in seat]
            bus_type = [i.text for i in bustype]

            min_length = min(len(orglist), len(depart_time), len(duration_t), len(arrival_time), len(amounts), len(ratings_k), len(seatavailable), len(bus_type))

            bus_details = []
            if min_length > 0:
                for index in range(min_length):
                    bus_details.append({
                        'orgname': orglist[index] if index < len(orglist) else None,
                        'depart_time': depart_time[index] if index < len(depart_time) else None,
                        "duration": duration_t[index] if index < len(duration_t) else None,
                        "arrival": arrival_time[index] if index < len(arrival_time) else None,
                        'amount': amounts[index] if index < len(amounts) else None,
                        "rating": ratings_k[index] if index < len(ratings_k) else None,
                        "seat": seatavailable[index] if index < len(seatavailable) else None,
                        "bustype": bus_type[index] if index < len(bus_type) else None,
                        "route_link": route_link  # Add route link here
                    })
            else:
                print("No data to process.")
            
            return bus_details
        except Exception as e:
            print(f"Error extracting bus details from route: {e}")
            return []

    # Scroll and load all elements on the page
    def scroll_and_load(driver, scroll_pause_time=2):
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    # Extract details for each route
    all_bus_details = []
    for link in route_links:
        print(f"Processing route: {link}")
        route_details = extract_bus_details(link)
        all_bus_details.extend(route_details)
        print(f"Extracted {len(route_details)} bus details from the route.")

    # Create a DataFrame and save to CSV
    df2 = pd.DataFrame(all_bus_details)
    print(df2)
    df2.to_csv('bus.csv', index=False)

finally:
    # Close the WebDriver
    driver.quit()


Error handling pagination: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pagination"}
  (Session info: chrome=127.0.6533.99); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71B4C9632+30946]
	(No symbol) [0x00007FF71B47E3C9]
	(No symbol) [0x00007FF71B376FDA]
	(No symbol) [0x00007FF71B3C822C]
	(No symbol) [0x00007FF71B3C850C]
	(No symbol) [0x00007FF71B40DCB7]
	(No symbol) [0x00007FF71B3ECAAF]
	(No symbol) [0x00007FF71B40B041]
	(No symbol) [0x00007FF71B3EC813]
	(No symbol) [0x00007FF71B3BA6E5]
	(No symbol) [0x00007FF71B3BB021]
	GetHandleVerifier [0x00007FF71B5FF83D+1301229]
	GetHandleVerifier [0x00007FF71B60BDB7+1351783]
	GetHandleVerifier [0x00007FF71B602A03+1313971]
	GetHandleVerifier [0x00007FF71B4FDD06+245686]
	(No symbol) [0x00007FF71B48758F]
	(No symbol) [0x00007FF71B483804]
	(No symbol) [0x00007FF71B48

In [18]:
data=pd.concat([df,df2],ignore_index=True)

In [19]:
df.to_csv('csv files/Allroute.csv',index = False)

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv(r'bus_details.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   orgname      861 non-null    object 
 1   depart_time  861 non-null    object 
 2   duration     861 non-null    object 
 3   arrival      861 non-null    object 
 4   amount       861 non-null    float64
 5   rating       861 non-null    float64
 6   seat         861 non-null    int64  
 7   bustype      861 non-null    object 
 8   route_link   861 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 60.7+ KB


##           MySQL Data storing

In [29]:
import mysql

In [60]:
import mysql.connector
import pandas as pd

In [61]:
df = pd.read_csv(r'E:\Project\Bus Project Automation\RedBusAutomation\csv files\Allroute.csv')

In [62]:
conn=mysql.connector.connect(host="localhost", user="root", password="password",database="busdetails")
my_cursor = conn.cursor()

In [63]:
create_table_sql = '''CREATE TABLE IF NOT EXISTS bus_routes(
                     Bus_name VARCHAR(255) NOT NULL,
                     Start_time VARCHAR(255) NOT NULL,
                     Total_duration VARCHAR(255) NOT NULL,
                     End_time VARCHAR(255) NOT NULL,
                     Price FLOAT NULL,
                     Ratings FLOAT NULL,
                     Seats_Available INT NOT NULL,
                     Bus_type VARCHAR(255) NOT NULL,
                     route_link VARCHAR(255) NOT NULL
                     )'''

# Execute table creation
my_cursor.execute(create_table_sql)

In [64]:
# Correct SQL with matching column names and placeholders


# SQL command for inserting data
insert_sql = """INSERT INTO bus_routes(
                    Bus_name,
                    Start_time,
                    Total_duration,
                    End_time,
                    Price,
                    Ratings,
                    Seats_Available,
                    Bus_type,
                    route_link)
                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

# Convert DataFrame to list of tuples
data = df.values.tolist()

# Execute batch insert
my_cursor.executemany(insert_sql, data)

# Commit the transaction using the connection object
conn.commit()

# Print number of rows inserted
print(my_cursor.rowcount, "rows were inserted.")

# Close the cursor and connection
my_cursor.close()
conn.close()





861 rows were inserted.


In [67]:
df1 = pd.read_csv(r'E:\Project\Bus Project Automation\RedBusAutomation\csv files\data.csv')

In [68]:
conn=mysql.connector.connect(host="localhost", user="root", password="password",database="Route")
my_cursor = conn.cursor()

In [69]:
create_table_sql = '''CREATE TABLE IF NOT EXISTS route(
                     RouteName VARCHAR(255) NOT NULL,
                     RouteLink VARCHAR(255) NOT NULL
                     )'''

# Execute table creation
my_cursor.execute(create_table_sql)

In [70]:
insert_sql = """INSERT INTO route(
                  routename,
                  routelink)
                VALUES(%s, %s)"""

# Convert DataFrame to list of tuples
data = df1.values.tolist()

# Execute batch insert
my_cursor.executemany(insert_sql, data)

# Commit the transaction using the connection object
conn.commit()

# Print number of rows inserted
print(my_cursor.rowcount, "rows were inserted.")

# Close the cursor and connection
my_cursor.close()
conn.close()

500 rows were inserted.
